In [ ]:
!pip install tensorflow_addons

In [2]:
import glob
import tensorflow as tf
import tensorflow_addons as tfa

from tqdm.notebook import tqdm
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.callbacks import EarlyStopping

%load_ext tensorboard

In [ ]:
#Clear
rm -rf ./logs/

**Data loader class**

In [3]:
class Data(object):

  def __init__(self, path):
      #Initial size of images == 150x150x3
      self.image_paths = path
      self.labels = self.get_labels(path)
  
  def get_labels(self, path):  
      classes = ['cabrio', 'kombi', 'sedan', 'hatchback', 'luxury-sport', 'SUV', 'pickup']
      labels = []
      for p in path:
        word_label = p.split('_')[0]
        word_label = word_label.split('/')
        labels.append(classes.index(word_label[-1]))
      return labels

  def parse_image(self, path, labels):
      image = tf.io.read_file(path) 
      image = tf.image.decode_png(image, channels=3)  
      image = tf.image.convert_image_dtype(image, tf.float32) 
      return image, labels
  
  def rotate_image(self, image, labels):
      image = tfa.image.rotate(image, 0.1745)
      image = tf.clip_by_value(image, 0.0, 1.0)
      return image, labels
  
  def flip_image(self, image, labels):
      image = tf.image.flip_left_right(image)
      return image, labels
  
  def dataset(self, batch_size, threads=4):
      dataset = tf.data.Dataset.from_tensor_slices((self.image_paths, self.labels))
      dataset = dataset.map(self.parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

      flip = dataset.map(self.flip_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
      dataset = dataset.concatenate(flip)

      rotate = dataset.map(self.rotate_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
      dataset = dataset.concatenate(rotate)

      dataset = dataset.shuffle(30000).batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)      
      return dataset

**Gather dataset and split into train/validation/test**

In [4]:
img_list = glob.glob('/content/drive/MyDrive/carsdata2/train/*.png')
rest_of_images = glob.glob('/content/drive/MyDrive/carsdata2/test2/*.png')

for i in rest_of_images:
  img_list.append(i)

log_dir = "logs/fit/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


BATCH_SIZE = 32

dataset = Data(img_list).dataset(BATCH_SIZE)

DATASET_SIZE = len(dataset)

train_size = int(0.8 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)
test_size = int(0.1 * DATASET_SIZE)


train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

**Neural network architecture - using ResNet**

In [ ]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_shape=(150, 150, 3))

headModel = baseModel.output
headModel = keras.layers.MaxPooling2D(pool_size=(2, 2))(headModel)
headModel = keras.layers.Flatten(name="flatten")(headModel)
headModel = keras.layers.Dense(256, activation="relu")(headModel)
headModel = keras.layers.Dropout(0.5)(headModel)
headModel = keras.layers.Dense(7, activation="softmax")(headModel)

model = keras.models.Model(inputs=baseModel.input, outputs=headModel)

model.compile(optimizer='adam',
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

In [6]:
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
model.summary()

In [ ]:
model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks=[es, tensorboard_callback])

In [ ]:
model.evaluate(test_dataset)

In [27]:
model.save('cars_resnet.h5')

In [ ]:
%tensorboard --logdir logs/fit